# Neural Networks

Se trata de un modelo clasificador/regresor que busca detectar patrones mediante la combinación de transformaciones de datos, usando las llamadas "neuronas".

ScitiLearn prove de modelos de redes neuronales para ML supervisado:
- https://scikit-learn.org/stable/modules/neural_networks_supervised.html

Y no supervisado:
- https://scikit-learn.org/stable/modules/neural_networks_unsupervised.html


En el caso supervisado, provee de modelos basados en *Multi-layer Perceptron*, que se pueden usar en clasificación y regresión.

Usaremos cñase `MLPRegressor` que Implementa un perceptrón de múltiples capas (MLP) 
- se entrena usando la retropropagación sin función de activación en la capa de salida (como si usara la *función de identidad* como función de activación).
- usa el *error cuadrado* como la función de pérdida, y la salida es un conjunto de valores continuos.

In [ ]:
import os

import pandas as pd
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import  r2_score
from sklearn.linear_model import  LinearRegression

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# carga y preprocesamiento de datos

df = pd.read_csv("https://github.com/ricardoahumada/Python_for_Data_Science/raw/refs/heads/master/data/2008_small.zip",nrows = 1000000)

df = df.dropna(subset =['AirTime','Distance','TaxiIn','TaxiOut'])
df = df.sample(frac=1).head(10000)

X = df[['AirTime','Distance','TaxiIn','TaxiOut']] 
Y = df['ArrDelay']

poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

X = pd.DataFrame(X_poly, columns = poly.get_feature_names_out(input_features=X.columns))


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=1)

scaler = StandardScaler()  
scaler.fit(X_train) # conocimiento

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# entrenamiento y score del modelo de NN

regr = MLPRegressor(solver='lbfgs', alpha=1, hidden_layer_sizes=(10,100,10)) #The ith element represents the number of neurons in the ith hidden layer.

model = regr.fit(X_train, y_train)

predictions = model.predict(X_test)
print("R cuadrado: ",r2_score(y_test, predictions))

>Comparamos con el modelo de regresión lineal

In [ ]:
regrLin = LinearRegression()
regrLin.fit(X_train,y_train)
y_pred = regrLin.predict(X_test)
print("R cuadrado: ",r2_score(y_test, y_pred))

>Algunos de los parámetros con los que podemos jugar en los modelos de red neurnal son:
>   
>- solver: {‘lbfgs’, ‘sgd’, ‘adam’}    
>- activation : {‘identity’, ‘logistic’, ‘tanh’, ‘relu’}
>- alpha : float, optional, default 0.0001 L2 penalty
>- learning_rate : {‘constant’, ‘invscaling’, ‘adaptive’}

>Considerar la cantidad de combinaciones posibles de parámetros...\
>https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html

### Selección de parámetros automatizada (sin funciones)

Más potente, más flexible, más control... pero requiere más código...

In [ ]:
# Modelo con 3 capas

alphas = [0.0000001,0.001,0.1]
layers = [2,5,10,20]
solvers = ["lbfgs","adam","sgd"]
listtodf = []
i = 0
print("Iterations:", len(alphas)*len(layers) ** 3 *len(solvers))

for alpha in alphas:
    for layer1 in layers: 
        for layer2 in layers:
            for layer3 in layers:
                for solver in solvers:
                    i += 1 # i = i + 1 // i++
                    regr = MLPRegressor(solver=solver, alpha=alpha, hidden_layer_sizes=(layer1,layer2,layer3),warm_start=True)
                    model = regr.fit(X_train, y_train)
                    predictions = model.predict(X_test)
                    print(i, 
                          "R cuadrado: ",round(r2_score(y_test, predictions),2), 
                          " Solver: ",solver, 
                          " Layer size: ", (layer1,layer2,layer3),
                          " Alpha: ",alpha)
                    listtodf.append([alpha,(layer1,layer2,layer3),solver,r2_score(y_test, predictions)])

In [ ]:
listtodf

>Obteniendo conclusiones sobre los datos

In [ ]:
parameters = pd.DataFrame(listtodf, columns =['Alphas', 'Layers','Solvers','R2']) 
parameters.to_csv("parameters.csv")

In [ ]:
parameters = pd.read_csv("parameters.csv",index_col = 0)
parameters

In [ ]:
parameters.groupby("Alphas").mean()["R2"].sort_values(ascending = False)

In [ ]:
parameters.groupby("Layers").mean()["R2"].sort_values(ascending = False)

In [ ]:
parameters.groupby("Solvers").mean()["R2"].sort_values(ascending = False)

In [ ]:
parameters[parameters.R2 == max(parameters.R2)]

## Ejercicio

Intenta clasificar/predecir lo mejor que puedas los datos de iris o algún dataset sencillo, usando NN. 

Puedes usar una de las columnas numéricas como respuesta (regresión) o la columa "species" (clasificación).

Aprovecha para añadir nuevas variables (feature engineering), seleccionar alguno de los parámetros que funcionan mejor con estos datos, etc.